# Data exoploration and cleaning

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load each raw CSV

In [124]:
# Italian weekly prices
fuel = pd.read_csv('../data/raw/prezzi_carburanti_settimanali.csv')

# Brent futures
brent = pd.read_csv('../data/raw/brent_oil_futures.csv')

# USD–EUR exchange rate
usd_eur = pd.read_csv('../data/raw/usd_eur.csv')

Check their basic structure of raw CSV

In [125]:
for name, df in [('Fuel', fuel), ('Brent', brent), ('FX', usd_eur)]:
    print(f'=== {name} ===')
    print(df.shape)
    print(df.columns)
    print(df.head(), '\n')

=== Fuel ===
(1050, 9)
Index(['DATA_RILEVAZIONE', 'BENZINA', 'GASOLIO_AUTO', 'GPL', 'METANO', 'GNL',
       'GASOLIO_RISCALDAMENTO', 'O.C._FLUIDO_BTZ', 'O.C._DENSO_BTZ'],
      dtype='object')
  DATA_RILEVAZIONE   BENZINA GASOLIO_AUTO     GPL METANO  GNL  \
0       2005-01-03  1.115,75     1.018,28   552,5    NaN  NaN   
1       2005-01-10     1.088     1.004,39  552,57    NaN  NaN   
2       2005-01-17  1.088,14     1.004,31  551,88    NaN  NaN   
3       2005-01-24  1.090,01     1.004,31  551,88    NaN  NaN   
4       2005-01-31  1.132,11      1.022,6  551,88    NaN  NaN   

  GASOLIO_RISCALDAMENTO O.C._FLUIDO_BTZ O.C._DENSO_BTZ  
0                 948,5          553,25         229,52  
1                947,94          554,22         238,37  
2                952,42          562,78         245,89  
3                963,98          573,42         243,84  
4                972,95          579,23         242,53   

=== Brent ===
(1088, 7)
Index(['Date', 'Price', 'Open', 'High', 'Low', '

Inspect missing values in each dataset

In [126]:
fuel.isna().sum()


DATA_RILEVAZIONE           0
BENZINA                    0
GASOLIO_AUTO               0
GPL                        0
METANO                   952
GNL                      952
GASOLIO_RISCALDAMENTO      0
O.C._FLUIDO_BTZ            0
O.C._DENSO_BTZ             0
dtype: int64

In [127]:
brent.isna().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
dtype: int64

In [128]:
usd_eur.isna().sum()


Date           0
Price          0
Open           0
High           0
Low            0
Vol.        1089
Change %       0
dtype: int64

# Cleaning and normalization

Cleaning dataset `fuel`

In [129]:
# Copy to avoid touching the raw
fuel_clean = fuel.copy()

# Convert date column
fuel_clean['DATA_RILEVAZIONE'] = pd.to_datetime(fuel_clean['DATA_RILEVAZIONE'])

# Rename columns for easier use later
fuel_clean.rename(columns={
    'DATA_RILEVAZIONE': 'date',
    'BENZINA': 'benzina',
    'GASOLIO_AUTO': 'gasolio'
}, inplace=True)

# Keep only what we need
fuel_clean = fuel_clean[['date', 'benzina', 'gasolio']]

# Convert values from string to float
for c in ['benzina', 'gasolio']:
    fuel_clean[c] = fuel_clean[c].astype(str).str.replace(',', '').astype(float)

fuel_clean.info()
fuel_clean.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     1050 non-null   datetime64[ns]
 1   benzina  1050 non-null   float64       
 2   gasolio  1050 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 24.7 KB


,date,benzina,gasolio
0,2005-01-03,1.11575,1.01828
1,2005-01-10,1.08800,1.00439
2,2005-01-17,1.08814,1.00431
3,2005-01-24,1.09001,1.00431
4,2005-01-31,1.13211,1.02260
5,2005-02-07,1.13587,1.02267
6,2005-02-14,1.13588,1.02246
7,2005-02-21,1.13604,1.02279
8,2005-02-28,1.13604,1.02285
9,2005-03-07,1.15878,1.04759


Cleaning dataset `brent`

In [130]:
brent_clean = brent.copy()

# Convert Date
brent_clean['Date'] = brent_clean['Date'].astype(str).str.strip()
brent_clean['Date'] = pd.to_datetime(brent_clean['Date'], dayfirst=False, errors='coerce')

# Rename colunms
brent_clean.rename(columns={'Date': 'date', 'Price': 'brent_usd'}, inplace=True)

brent_clean['brent_usd'] = (
    brent_clean['brent_usd']
    .astype(str)
    .str.replace(',', '')
    .astype(float)
)

# Keep only date and price
brent_clean = brent_clean[['date', 'brent_usd']]


brent_clean.sort_values('date', inplace=True)

brent_clean.info()
brent_clean.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 1088 entries, 1087 to 0
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       1088 non-null   datetime64[ns]
 1   brent_usd  1088 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 25.5 KB


,date,brent_usd
1087,2005-01-02,43.18
1086,2005-01-09,45.15
1085,2005-01-16,45.73
1084,2005-01-23,44.95
1083,2005-01-30,43.89
1082,2005-02-06,44.80
1081,2005-02-13,46.34
1080,2005-02-20,49.61
1079,2005-02-27,51.80
1078,2005-03-06,53.10


Cleaning dataset `USD–EUR dataset`

In [131]:
usd_eur_clean = usd_eur.copy()

# Convert Date
usd_eur_clean['Date'] = usd_eur_clean['Date'].astype(str).str.strip()
usd_eur_clean['Date'] = pd.to_datetime(usd_eur_clean['Date'], dayfirst=False, errors='coerce')

# Rename colunms
usd_eur_clean.rename(columns={'Date': 'date', 'Price': 'usd_eur'}, inplace=True)

usd_eur_clean['usd_eur'] = (
    usd_eur_clean['usd_eur']
    .astype(str)
    .str.replace(',', '')
    .astype(float)
)

# Keep only date and price
usd_eur_clean = usd_eur_clean[['date', 'usd_eur']]


usd_eur_clean.sort_values('date', inplace=True)

usd_eur_clean.info()
usd_eur_clean.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     1089 non-null   datetime64[ns]
 1   usd_eur  1089 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 17.1 KB


,date,usd_eur
0,2005-01-02,0.7660
1,2005-01-09,0.7630
2,2005-01-16,0.7664
3,2005-01-23,0.7667
4,2005-01-30,0.7768
5,2005-02-06,0.7774
6,2005-02-13,0.7651
7,2005-02-20,0.7551
8,2005-02-27,0.7552
9,2005-03-06,0.7433


Save the cleaned versions

In [132]:
fuel_clean.to_csv('../data/processed/fuel_clean.csv', index=False)
brent_clean.to_csv('../data/processed/brent_clean.csv', index=False)
usd_eur_clean.to_csv('../data/processed/usd_eur_clean.csv', index=False)